In [2]:
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, ConversationHandler
from pycaret.classification import *

In [3]:
# global estado, mapResult

In [4]:
def sendHello(update: Update, context: CallbackContext):
    nome = update['message']['chat']['first_name']
    update.message.reply_text(f"Olá {nome}")

def infoBot(update: Update, context: CallbackContext):
    info = '''Eu sou um Bot com a finalidade de auxiliar na detecção de diabetes, criado pelo estudante Flávio Correia da UFC de Russas.'''
    update.message.reply_text(f"{info}")

In [5]:
def resetState():
    global estado, mapResult
    estado = ""
    mapResult = {}

In [6]:
def convertNumeric( strNum ):
    if str(strNum).replace(",", ".").replace(".", "", 1).isdigit():
        return float( strNum.replace(",", ".") )
    return None

In [7]:
# funcao inicial do bot
def startDoctor(update: Update, context: CallbackContext): 
    global estado
    resetState()
    
    update.message.reply_text(f"Informe a quantidade de vezes grávida:")
    estado = "numPreg"

In [8]:
def startAllQuestions(update: Update, context: CallbackContext): 
    global estado
    resetState()
    
    update.message.reply_text(f"Informe a função de linhagem de diabetes:")
    estado = "pedigreeF"

In [9]:
def messageFunctionMain(update: Update, context: CallbackContext):
    global estado, mapResult
    
    message = update['message']['text']
    value = convertNumeric(message)
    '''
    if estado == "pedigreeF":
        if value:
            mapResult["pedigreeF"] = value
            update.message.reply_text(f"Informe a quantidade de vezes grávida:")
            estado = "numPreg"
        else:
            update.message.reply_text(f"Função de linhagem de diabetes {message} inválida, informe novamente:")
    '''
    if estado == "numPreg":
        if value:
            mapResult["numPreg"] = value
            update.message.reply_text(f"Informe a taxa de glicose:")
            estado = "glic"            
        else:
            update.message.reply_text(f"Quantidade de vezes grávida {message} inválida, informe novamente:")       
        
    elif estado == "glic":
        if value:
            mapResult["glic"] = value
            update.message.reply_text(f"Informe a pressão arterial diastólica (mmHg):")
            estado = "blodp"            
        else:
            update.message.reply_text(f"Taxa de glicose {message} inválida, informe novamente:")         
    
    elif estado == "blodp":
        if value:
            mapResult["blodp"] = value
            #update.message.reply_text(f"Informe a espessura da dobra cutânea do tríceps (mm):")
            update.message.reply_text(f"Informe a função de linhagem de diabetes:")
            estado = "pedigreeF" #  skin        
        else:            
            update.message.reply_text(f"Pressão arterial diastólica {message} inválida, informe novamente:")   
    
    #'''
    # update.message.reply_text(f"Informe a função de linhagem de diabetes:")
    # estado = "pedigreeF"
    elif estado == "pedigreeF":
        if value:
            mapResult["pedigreeF"] = value
            update.message.reply_text(f"Informe a espessura da dobra cutânea do tríceps (mm):")
            estado = "skin"            
        else:
            update.message.reply_text(f"Função de linhagem {message} inválida, informe novamente:") 
    #'''
    
    elif estado == "skin":
        if value:
            mapResult["skin"] = value
            update.message.reply_text(f"Informe o valor da insulina:")
            estado = "insulin"            
        else:
            update.message.reply_text(f"Espessura de pele {message} inválida, informe novamente:")
                
        
    elif estado == "insulin":
        if value:
            mapResult["insulin"] = value
            update.message.reply_text(f"Informe o índice de massa corporal:")
            estado = "imc"            
        else:
            update.message.reply_text(f"Valor de insulina {message} inválida, informe novamente:")         
    
    elif estado == "imc":
        if value:
            mapResult["imc"] = value
            update.message.reply_text(f"Informe a idade:")
            estado = "age"            
        else:
            update.message.reply_text(f"Valor de massa corporal {message} inválido, informe novamente:") 
        
    elif estado == "age":
        if value:
            mapResult["age"] = value
            
            response = getPrediction( mapResult )
            score = response.Score
            score = round( score*100, 2 )
            if response.Label == 0:
                update.message.reply_text(f"Resultado falso para diabetes com probabilidade de {score}%.")
            else:
                update.message.reply_text(f"Resultado positivo para diabetes com probabilidade de {score}%.")
                
            estado = "."            
        else:
            update.message.reply_text(f"Valor da idade {message} inválida, informe novamente:")
    
    elif estado == "":
        update.message.reply_text(f"Estado resetado.")

In [10]:
'''
COMANDOS CADASTRADOS NÃO SÃO INTERPRETADOS COMO TEXTO
    MAS COMANDOS NÃO CADASTRADOS ATIVAM O MANIPULADOR DE TEXTO
''';

In [11]:
mx = load_model('xgboostM8136')

Transformation Pipeline and Model Successfully Loaded


In [12]:
def getPrediction( info ):
    dfInfo = pd.DataFrame( [ info ] )
    
    mapColumns = {
        'numPreg':'Pregnancies', 'glic':'Glucose', 'blodp':'BloodPressure', 
        'skin':'SkinThickness', 'insulin':'Insulin', 'imc':'BMI', 
        'pedigreeF':'DiabetesPedigreeFunction', 'age':'Age'
    }
    
    dfData = dfInfo.rename( columns=mapColumns )
                          
    return predict_model( mx , data=dfData ).loc[0, ['Label', 'Score']]

In [13]:
mapE = {'numPreg': 1.0, 'glic': 18.0, 'blodp': 72.0, 'skin': 35.0, 
        'insulin': 400, 'imc': 33.0, 'pedigreeF': 0.7,  'age': 50.0 }

dfPred = pd.DataFrame( [ mapE.values() ], columns =  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
       'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] )
dfPred

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1.0,18.0,72.0,35.0,400,33.0,0.7,50.0


In [14]:
getPrediction( mapE )

Label    0.0000
Score    0.6703
Name: 0, dtype: float64

In [15]:
r = getPrediction( mapE )

In [16]:
estado = ""
mapResult = {}

In [ ]:
TOKEN = "5579725859:AAHJrLmRHiHc1M5SkBPxif8dHShRgYqYUGs"
bot = Updater(TOKEN)

#MANIPULADOR DE ROTAS
dispatcher = bot.dispatcher

#ADICIONA ROTAS
dispatcher.add_handler(CommandHandler("ola", sendHello))
dispatcher.add_handler(CommandHandler("iniciar", startDoctor))
#dispatcher.add_handler(CommandHandler("startAll", startAllQuestions))
dispatcher.add_handler(CommandHandler("info", infoBot))

# Manipulador de mensagens de texto 
dispatcher.add_handler(MessageHandler( Filters.text, messageFunctionMain))

#INICIA O BOT
bot.start_polling()
print("Bot Iniciado")

#PERMITE CANCELAR O BOT
bot.idle()

Bot Iniciado


In [19]:
estado

'skin'

In [20]:
mapResult

{'numPreg': 6.0, 'glic': 148.0, 'blodp': 72.0, 'pedigreeF': 0.627}

In [9]:
import pandas as pd

In [15]:
l = [32,2,2,8,2,2,1,2,4,4,8,8,4,1,32,16,4,4,4,4,2,2,8,2,4,2,2,8,4,4,4,4,4,4]
sum(l), sum(l)/8 

(198, 24.75)

In [10]:
pd.Series(l).value_counts()

4     14
2     10
8      5
32     2
1      2
16     1
dtype: int64

In [13]:
25*1.6

40.0

# EXEMPLO MAP PEGUE MSG

In [24]:
up = {'update_id': 140445672, 'message': {'message_id': 24, 'delete_chat_photo': False, 'chat': {'last_name': 'Correia', 'id': 1022021295, 'type': 'private', 'first_name': 'Flávio'}, 'entities': [], 'new_chat_photo': [], 'date': 1660747354, 'supergroup_chat_created': False, 'photo': [], 'new_chat_members': [], 'group_chat_created': False, 'text': 'Senhaqwe', 'caption_entities': [], 'channel_chat_created': False, 'from': {'is_bot': False, 'language_code': 'pt-br', 'last_name': 'Correia', 'first_name': 'Flávio', 'id': 1022021295}}}

In [29]:
update = {'message': {'delete_chat_photo': False, 'group_chat_created': False, 'text': '/send', 'new_chat_members': [], 'chat': {'type': 'private', 'first_name': 'Flávio', 'last_name': 'Correia', 'id': 1022021295}, 'message_id': 12, 'caption_entities': [], 'channel_chat_created': False, 'new_chat_photo': [], 'photo': [], 'entities': [{'length': 5, 'offset': 0, 'type': 'bot_command'}], 'date': 1660260499, 'supergroup_chat_created': False, 'from': {'first_name': 'Flávio', 'last_name': 'Correia', 'language_code': 'en', 'is_bot': False, 'id': 1022021295}}, 'update_id': 140445663}

In [32]:
update['message']['from']['first_name']

'Flávio'

In [ ]:
##testeBotMain.py
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from botFunctions import *

TOKEN = "5321007840:AAEKo1UpT4fosWtyOabU5unKfqz2YRRNCQ0"
bot = Updater(TOKEN)

#MANIPULADOR DE ROTAS
dispatcher = bot.dispatcher

#ADICIONA ROTAS
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(CommandHandler("ola", teste2))


#INICIA O BOT
print("Bot Iniciado")
bot.start_polling()

#PERMITE CANCELAR O BOT
bot.idle()

'''
dispatcher.add_handler(MessageHandler(Filters.group, self._group_hook()))
dispatcher.add_handler(MessageHandler(Filters.text, self._msg_hook(TextMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.photo, self._msg_hook(PhotoMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.video, self._msg_hook(VideoMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.contact, self._msg_hook(ContactMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.location, self._msg_hook(LocationMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.document, self._msg_hook(DocumentMessageEvent)))
dispatcher.add_handler(MessageHandler(Filters.command, self._command_hook()))
'''


In [ ]:
## botFunctions.py
#C:\\Users\\FL20310106\\AppData\\Local\\Continuum\\anaconda3\\envs\\pydev
#C:\Users\FL20310106\AppData\Local\Continuum\anaconda3\envs\pyDev
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from folderManagement import *
from botPandas import *
from botMachineLearning import *
#update.message.reply_text("MENSAGEM")

id_image = 0

#MSG QUANDO BOT É INICIADO
def start(update: Update, context: CallbackContext):
    update.message.reply_text("Você acionou start")


def detectFaceImage(update: Update, context: CallbackContext):    
    global id_image
    update.message.reply_text("Imagem recebida para detecção.")
    file = update.message.photo[-1].get_file()
    chatId = update.message.chat.id

    pathImage = f"faceDetection//{chatId}_image.jpg"
    pathImage = f"faceDetection//{chatId}_image_{id_image}.jpg"
    removeFile(pathImage)
    file.download(pathImage)

    statusDetection = detectFaceCv2( pathImage )
    if statusDetection == "-1": # nao detectou faces
        update.message.reply_text("Não foram detectadas faces.")
    else: # detectou faces
        update.message.reply_text("Foram detectados rostos.")
        context.bot.send_photo(chat_id=update.message.chat.id, photo=open(statusDetection, 'rb'))
        
    id_image +=1

def downloadFile(update: Update, context: CallbackContext):  
    file = update.message.document
    extensao = file.file_name.split(".")[-1]
    if extensao in ["csv", "xlsx", "xls"]:
        update.message.reply_text("Arquivo recebido")
        clearFolder("downloads")

        #FAZ DOWNLOAD DO ARQUIVO
        with open("downloads/"+file.file_name, 'wb') as f:
            context.bot.get_file(update.message.document).download(out=f)
        
        if extensao in ["xlsx", "xls"]:
            update.message.reply_text("Excel")
            describeDataFrame( pd.read_excel("downloads/"+file.file_name) )
        elif extensao in ["csv"]:
            update.message.reply_text("CSV")
            describeDataFrame( pd.read_csv("downloads/"+file.file_name, sep=";") )
        
        update.message.reply_text("Enviando resumo")
        #ENVIA IMAGEM PARA O USUARIO
        context.bot.send_photo(chat_id=update.message.chat.id, photo=open('created_files/df.png', 'rb'))
    else:
        update.message.reply_text("Tipo de arquivo não suportado.")

def sendXLSXFile(update: Update, context: CallbackContext): 
    context.bot.sendDocument(
        chat_id=update.message.chat.id, 
        caption ="Arquivo excel do relatório.", 
        document=open("created_files/df.xlsx", 'rb'))

def teste1(update: Update, context: CallbackContext):
    print(update)
    print("- - - - - "*5)
    print(context)
    update.message.reply_text('Olá')

def teste2(update: Update, context: CallbackContext):
    user = update.effective_user
    update.message.reply_markdown_v2(
        f'Olá {user.mention_markdown_v2()}', reply_markup=ForceReply(selective=True),
    )



'''
ENVIAR DOCUMENTO PELO TELEGRAM
def send_document(update, context):
    chat_id = update.message.chat_id
    document = open('image1.png', 'rb')
    context.bot.send_document(chat_id, document)

dispatcher.add_handler(CommandHandler("send", send_document))
'''

In [ ]:
## botMachineLearning.py
import cv2
import matplotlib.pyplot as plt

#more weights
#https://github.com/opencv/opencv/tree/master/data/haarcascades
cv2FaceDetector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def detectFaceCv2( imagePath ):
    #img = cv2.cvtColor( cv2.imread( imagePath ), cv2.COLOR_BGR2RGB)
    img = cv2.imread( imagePath )
    img_gray = cv2.imread(imagePath, 0)

    faces = cv2FaceDetector.detectMultiScale(img_gray)

    if len(faces) > 0:
        for x, y, width, height in faces:
            cv2.rectangle(img, (x, y), (x + width, y + height), color=(255, 0, 0), thickness=2)

        imagePathDetected = imagePath.split(".")[0]+"_faces.jpg"
        cv2.imwrite(imagePathDetected, img);
        return imagePathDetected
    else:
        return "-1"


In [ ]:
## botPandas
import pandas as pd
import dataframe_image as dfi

def describeDataFrame( df:pd.DataFrame ):
    #return df.describe(exclude=[object]) 
    #return df.describe(include="all")
    df_final = df.describe(include="all").round(2).fillna(0)
    df_final = df_final.loc[ ["count", "unique", "mean", "std", "min", "25%", "50%", "75%", "max"] ,:]
    dfi.export( df_final, "created_files/df.png" )
    df_final.to_excel( "created_files/df.xlsx" )
    return df_final

In [ ]:
## folderManagement
import os

def clearFolder( folderName:str ):
    listFiles = os.listdir( folderName )
    for file in listFiles:
        os.remove(folderName+"/"+file)

def removeFile( fileName:str , path:str="./"):
    listFiles = os.listdir( path )    
    if fileName in listFiles:
        os.remove(path+"/"+fileName)


In [ ]:
- Camila do Rocio Nardin - Legal
- Claudia Bravo Quezada - +-